# Hipótese do Arquivo Matches

Este notebook será utilizado para apresentar e testar uma hipótese baseada nos dados da tabela **Matches** fornecida pela IBM, que contém informações detalhadas sobre partidas de futebol, incluindo times, resultados e estatísticas associadas. O foco principal é analisar os dados e validar uma hipótese específica relacionada ao desempenho dos times.

## Objetivo

O objetivo deste notebook é estruturar a hipótese de maneira clara e realizar análises que possam confirmá-la ou refutá-la, utilizando métodos estatísticos e exploratórios.

## Como Usar Este Notebook

1. **Configuração do Ambiente**:
   - **Google Colab**: No Google Colab, faça o upload das tabelas para o Google Drive e monte o drive no notebook.
   - **Localmente**: Se for rodar localmente, ajuste os caminhos dos arquivos ou coloque-os no mesmo diretório do notebook.

2. **Instalação de Dependências**:
   - Instale todas as bibliotecas necessárias com o comando:
     ```python
     !pip install -r requirements.txt
     ```

3. **Execução do Notebook**:
   - Execute as células de código na sequência para garantir a correta análise dos dados.

## Nesse Notebook Será Abordado

1. **Formulação e Teste de Hipóteses**:
   - Definição de uma hipótese com base nos dados.
   - Teste da hipótese utilizando métodos estatísticos.
   - Análise dos resultados para validar ou refutar a hipótese.


# Dependências
Para rodar o notebook de forma local, é recomendado que inicie uma venv (ambiente virtual) e instale as dependências.

Se estiver utilizando o Google Colab, pule esta etapa.

# Importando bibliotecas

Aqui é importado as dependências necessárias para a executação do projeto.

In [3]:
import pandas as pd #para ler, visualizar e printar infos do df
import matplotlib.pyplot as plt #para construir e customizar gráficos
import seaborn as sns #para visualizar uns gráficos
import numpy as np #numpy porque é sempre bom importar numpy

# Carregando o dataset

Feita a importação do arquivo para leitura dos dados.

In [ ]:
# Carregar os dados das partidas
df = pd.read_csv('../../notebooks/data/tratado/matches_tratado.csv')

# 1. Hipóteses

### Hipótese 1 (Principal utilizada no modelo preditivo)
### Probabilidade de qual time vai ganhar
#### Time da casa

In [ ]:
# Exemplo de dados hipotéticos de probabilidade para os times (previsão feita anteriormente)
times = ['Flamengo', 'Palmeiras']
probabilidades = [0.65, 0.35]  # Supondo que o Flamengo tenha 65% de chance de ganhar e o Palmeiras 35%

# Criar o gráfico de barras
plt.figure(figsize=(8, 6))
plt.bar(times, probabilidades, color=['red', 'green'])

# Adicionar títulos e rótulos
plt.title('Probabilidade de Vitória - Flamengo x Palmeiras')
plt.xlabel('Times')
plt.ylabel('Probabilidade de Vitória')

# Mostrar as porcentagens no topo das barras
for i, v in enumerate(probabilidades):
    plt.text(i, v + 0.01, f'{v*100:.1f}%', ha='center', fontsize=12)

plt.ylim(0, 1)  # Limitar o eixo y de 0 a 1 para representar porcentagens
plt.show()

### Análise:

**Observação:** Nessa hipótese foi utilizado como exemplo o time Flamengo e o Palmeiras, com a variável probabilidade imputada manualmente, tudo apenas para análise.

**Probabilidade de Vitória:**
  - **Flamengo**: A barra do Flamengo indica uma probabilidade de 65% de vitória, sugerindo que, com base nas análises anteriores, o time tem maior chance de ganhar a partida.
  - **Palmeiras**: A barra do Palmeiras mostra uma probabilidade de 35%, indicando que este time tem uma chance menor de vencer o confronto em comparação com o Flamengo.

<br>

### ***Conclusões:***

O gráfico apresenta uma clara diferença nas probabilidades de vitória entre os dois times. As principais observações incluem:

* **Flamengo**: Com uma probabilidade de 65%, o Flamengo é apontado como o favorito para vencer a partida.
* **Palmeiras**: O Palmeiras, com 35% de chance, é visto como o time menos provável de ganhar este confronto específico.
* A visualização sugere uma vantagem do Flamengo em termos de probabilidade, porém não garante o resultado final, apenas reflete a tendência com base nos dados analisados.

Essa hipótese representa um exemplo do tipo de previsão que nosso modelo irá realizar. O modelo preditivo será utilizado para estimar a probabilidade de vitória de diferentes times com base em dados históricos e análises preditivas, fornecendo insights semelhantes aos que foram demonstrados neste gráfico.


### Hipótese 2
### Equipes que sofrem mais escanteios tendem a ter um menor número de vitórias

In [ ]:
df['result'] = np.where(df['home_team_goal_count'] > df['away_team_goal_count'], 'home_win',
                           np.where(df['home_team_goal_count'] < df['away_team_goal_count'], 'away_win', 'tie'))


# Supondo que os valores reais na coluna 'result' sejam, por exemplo, 'away_win', 'tie', 'home_win':
df['Resultado'] = df['result'].map({
    'away_win': 'Vitória Visitante',
    'tie': 'Empate',
    'home_win': 'Vitória da Casa'
})

# Criar a coluna 'Escanteios'
df['Escanteios'] = df['home_team_corner_count'] + df['away_team_corner_count']

# Agora podemos aplicar o melt com segurança
df_melted = df.melt(id_vars=['Resultado', 'Escanteios'], 
                    value_vars=['home_team_corner_count', 'away_team_corner_count'],
                    var_name='Time', value_name='Escanteios_Individuais')

# Substituir os nomes para 'Casa' e 'Visitante' na coluna 'Time'
df_melted['Time'] = df_melted['Time'].map({
    'home_team_corner_count': 'Casa',
    'away_team_corner_count': 'Visitante'
})

# Criar o plot
sns.violinplot(x='Resultado', y='Escanteios_Individuais', hue='Time', data=df_melted, split=True)
plt.title('Relação entre Escanteios e Resultado da Partida')
plt.xlabel('Resultado da Partida')
plt.ylabel('Número de Escanteios')
plt.legend(title='Time', loc='upper left')
plt.show()

### Análise:

**Vitória da Casa:**
        O gráfico mostra que, em jogos em que o time da casa venceu, a distribuição dos escanteios está ligeiramente mais alta para o time da casa em comparação ao time visitante. Isso sugere que, na maioria das vezes, o time da casa conseguiu mais escanteios em partidas que venceu.

**Empate:**
        No caso de empates, a distribuição dos escanteios é mais similar entre os dois times, o que é esperado, dado que o resultado da partida foi equilibrado.

**Vitória Visitante:**
        Aqui, a distribuição dos escanteios tende a ser mais elevada para o time visitante, o que indica que, em jogos onde o time visitante venceu, ele frequentemente obteve mais escanteios que o time da casa.

<br>

### ***Conclusões:***

* **Escanteios como Indicador de Dominância:** Este gráfico pode sugerir que o número de escanteios pode estar relacionado com o domínio de um time durante o jogo, que muitas vezes se reflete no resultado final. Times que dominam em escanteios também tendem a vencer o jogo, seja em casa ou fora.

* **Equilíbrio em Empates:** Em jogos que terminam em empate, os números de escanteios para ambos os times são mais equilibrados, reforçando a ideia de que esses jogos foram mais equilibrados em termos de oportunidades.

<br>

&nbsp;&nbsp;&nbsp;&nbsp;Este gráfico visualiza bem a relação entre a quantidade de escanteios e o resultado das partidas, destacando como essa métrica pode estar associada ao sucesso de um time em termos de vitória ou derrota.

### Hipótese 3

In [ ]:
# Transformando os dados para o formato longo
df_melted = pd.melt(df, id_vars='result', value_vars=['home_team_red_cards', 'away_team_red_cards'],
                    var_name='Time', value_name='Cartões Vermelhos')

# Mapeando os valores de 'Time' para etiquetas mais legíveis
df_melted['Time'] = df_melted['Time'].map({'home_team_red_cards': 'Time da Casa', 'away_team_red_cards': 'Time Adversário'})

# Criando o Violin Plot
plt.figure(figsize=(10, 6))
sns.violinplot(x='result', y='Cartões Vermelhos', hue='Time', data=df_melted, split=True)

plt.title('Relação entre Cartões Vermelhos e Resultado da Partida')
plt.xlabel('Resultado da Partida')
plt.ylabel('Número de Cartões Vermelhos')
plt.grid(True)
plt.legend(title='Time', loc='upper right')
plt.show()

### Análise:

**Vitória do Time da Casa (home_win):**
        Observa-se que, em geral, o time da casa tende a receber poucos ou nenhum cartão vermelho quando vence a partida. A distribuição é mais larga perto de zero para os cartões vermelhos, indicando que, na maioria das vitórias da casa, poucos cartões vermelhos foram recebidos.
        O time visitante também apresenta uma tendência similar, mas com uma leve inclinação para mais cartões vermelhos em algumas vitórias do time da casa.

**Empate (tie):**
        Em jogos que terminam empatados, tanto o time da casa quanto o visitante tendem a ter uma distribuição mais uniforme de cartões vermelhos, embora ainda seja mais comum que ambos os times recebam poucos cartões vermelhos.

**Vitória do Time Visitante (away_win):**
        O padrão se repete, onde os times da casa tendem a ter menos cartões vermelhos em jogos que perdem.
        No entanto, o time visitante, ao vencer, parece ter uma leve distribuição de mais cartões vermelhos, sugerindo que mesmo com mais cartões, eles ainda podem sair vitoriosos.

<br>

### ***Conclusões:***

A maioria dos jogos, independentemente do resultado, tende a envolver poucos cartões vermelhos, com algumas exceções.
Não há uma correlação muito forte visível entre o número de cartões vermelhos e as derrotas, mas parece haver uma tendência de que times que perdem (seja em casa ou fora) não recebam tantos cartões vermelhos como o esperado, ou que mesmo recebendo, isso não afeta diretamente o resultado da vitória ou derrota.

<br>

Este gráfico sugere que, embora os cartões vermelhos sejam um fator de desvantagem, eles não são necessariamente determinantes para o resultado da partida, pelo menos no contexto dos dados apresentados.

# 2. Referências

Está é uma seção de referências com relação as bibliotecas que utilizamos ao longo deste arquivo

NUMPY. NumPy Documentation. Disponível em: <https://numpy.org/doc/>.

‌PANDAS. pandas documentation. Disponível em: <https://pandas.pydata.org/docs/>.

MATPLOTLIB. Matplotlib: Python plotting — Matplotlib 3.3.4 documentation. Disponível em: <https://matplotlib.org/stable/index.html>.

‌SEABORN. seaborn: statistical data visualization — seaborn 0.9.0 documentation. Disponível em: <https://seaborn.pydata.org/>.

‌SciPy documentation — SciPy v1.8.1 Manual. Disponível em: <https://docs.scipy.org/doc/scipy/>.

‌PYTHON SOFTWARE FOUNDATION. Math — Mathematical Functions — Python 3.8.3rc1 Documentation. Disponível em: <https://docs.python.org/3/library/math.html>.

‌SCIKIT-LEARN. scikit-learn: machine learning in Python. Disponível em: <https://scikit-learn.org/stable/>.
